In [1]:
import re
import sys
from random import random
from math import log
import numpy as np
from collections import defaultdict
import collections
import time

In [2]:
file_en = 'data/training.en'
file_de = 'data/training.de'
file_es = 'data/training.es'
file_mo = 'data/model-br.en'
file_test = 'data/test'

In [3]:

option =  set(chr(i) for i in range(97,123))
option =  option|{" ",".","#","0"}
print(len(option))

full = set()
for i in option:
    for j in option:
        for k in option:
            full = full|{i+j+k}

x_e_y = set()
for i in option:
        for j in option:
            x_e_y = x_e_y|{i+"#"+j}

e_x_e = set()
for i in option:
    e_x_e = e_x_e|{"#"+i+"#"}
                                          
e_e_x = set()
for i in (option-{"#"}):
    e_e_x = e_e_x|{"#"+"#"+i}
rest = full - x_e_y - e_x_e | e_e_x


30


In [6]:
#task1#
#Set = [chr(i) for i in range(65,91) and range()]

Set = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
      'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
def preprocess_line(line):
    res = ""
    for i in line.rstrip("\n"):
        if i in Set:
            res += i.lower()
        elif i.isspace():
            res += ' '
        elif i.isdigit():
            res += "0"
        elif i == '.':
            res += '.'
    return res
string = "What is Your Name123?.!?"
preprocess_line(string)


'what is your name000.'

In [7]:

#task3#
def generate_LM(infile,full):
    tri_counts = defaultdict(list)
    his_counts = defaultdict(list)
    #rest = full
    with open(infile) as f:
        for line in f:
            line = preprocess_line(line)
            line = "##" + line + "#"
            for j in range(len(line)-(2)):
                trigram = line[j:j+3]
                his = trigram[:2]
                try:
                    his_counts[his][0] += 1
                except:
                    his_counts[his].append(1)

                try:
                    tri_counts[trigram][0] += 1
                except:
                    tri_counts[trigram].append(1.005)
                    his_counts[trigram[0:2]][0] += 0.005
                    tri_counts[trigram].append(his)
                    his_counts[his].append(trigram)
                #if trigram in rest:
                    #rest.remove(trigram)
                full = full - {trigram}
    for trigram in full:
        his = trigram[:2]
        try:
            his_counts[his][0] += 0.005
        except:
            his_counts[his].append(0.005)

        tri_counts[trigram].append(0.005)
        tri_counts[trigram].append(trigram[0:2])
        his_counts[his].append(trigram)

    for s in sorted(tri_counts.keys()):
        tri_counts[s].append(tri_counts[s][0]/his_counts[tri_counts[s][1]][0])
    print(len(full))
    return tri_counts, his_counts


In [8]:
def load_data_into(infile):
    with open(infile) as f:
        tri_counts_model = defaultdict(list)
        his_counts_model = defaultdict(list)
        for line in f:
            tab = line[0:3]
            p = int(line[11:13])
            prob = float(line[4:9])*pow(10,-p)
            tri_counts_model[tab].append(1)
            tri_counts_model[tab].append(tab[0:2])
            tri_counts_model[tab].append(prob)
            try:
                his_counts_model[tab[0:2]][0] += 1
            except:
                his_counts_model[tab[0:2]].append(1)
            his_counts_model[tab[0:2]].append(tab)
    return tri_counts_model,his_counts_model
        

In [16]:
#task4#
def generate_from_LM(tri_counts, his_counts):
    key = "##"
    para = key
    for i in range(300):
        opt = list(his_counts[key][1:])
        try:
            prob = list(tri_counts[key][2] for key in opt)
            bins = np.cumsum(prob)
            choose = np.digitize(np.random.rand(1), bins)
            key = opt[choose[0]]
        
        except:
            print("can not consist 300 character with prob")
            break      
        para += key[2]
        key = key[1:]
    return para

In [10]:
start = time.time()
tri_counts,his_counts = generate_LM(file_en,rest)
tri_de_counts,his_de_counts = generate_LM(file_de,rest)
tri_es_counts,his_es_counts = generate_LM(file_es,rest)
tri_counts_model,his_counts_model = load_data_into(file_mo)
end = time.time()
print("{:.2f}s".format(end-start))

22486
22006
22867
385.35s


In [17]:
String = generate_from_LM(tri_counts,his_counts)
print(String)
#tri_counts_model,his_counts_model = load_data_into(file_mo)

String_model = generate_from_LM(tri_counts_model,his_counts_model)
print(String_model)



can not consist 300 character with prob
##mr on amis letwo re it extres ar to ot the ne onsidessuctur  burectultiamentaill pare acce submissin an entate of thatexandints cris red be cominabill rabse 0.#
can not consist 300 character with prob
##do heah boo.#


In [25]:
#task5#
 
def compute_perplexity(inFile,tri_counts):
    with open(inFile) as f:
        n = 0
        pw = 0
        for line in f:
            line = preprocess_line(line)
            line = "##"+line+"#"  
            n += len(line)-2
            for j in range(len(line)-(2)):
                temp = line[j:j+3]
                ta = tri_counts[temp][2]
                pw += log(tri_counts[temp][2],2)         
        hm = -pw/n
        per = pow(2,hm)
    return per
    

In [234]:
perp_en = compute_perplexity(file_test,tri_counts)
perp_model = compute_perplexity(file_test,tri_counts_model)
perp_es = compute_perplexity(file_test,tri_es_counts)
perp_de = compute_perplexity(file_test,tri_de_counts)
print(perp_en)
print(perp_model)
print(perp_es)
print(perp_de)

9.900934286409948
22.094457902881697
47.55191657771604
47.318787919289704
